In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import xarray as xr

In [4]:
from neurodsp.plts.time_series import plot_time_series
from ripple_detection.core import gaussian_smooth
smoothing_sigma = 2 # seconds

In [5]:
from ecephys.data import channel_groups, paths
from ecephys.sglx_utils import load_timeseries
from ecephys.signal.timefrequency import parallel_spectrogram_welch
from ecephys.plot import plot_spectrogram, plot_hypnogram_overlay
from ecephys.scoring import load_visbrain_hypnogram

In [100]:
SUBJECT="Segundo"
CONDITION="REC-2-0"
START_TIME = 0
END_TIME = None
bin_path = paths.get_datapath(subject=SUBJECT, condition=CONDITION, data="lf.bin")

In [101]:
delta_range = (0.5, 4)
theta_range = (5, 10)

In [102]:
def get_spectrogram(sig):
    freqs, spg_times, spg = parallel_spectrogram_welch(sig.values, sig.fs)
    spg_times = spg_times + START_TIME
    spg = xr.DataArray(spg, dims=("frequency", "time", "channel"), coords={"frequency": freqs, "time": spg_times, "channel": sig.channel.values})
    return spg.median(dim="channel")
    
def _get_bandpower(spg, f_range, chans):
    bandpower = spg.sel(frequency=slice(*f_range)).sum(dim="frequency")
    bandpower.attrs['long_name'] = "{} Hz PSD".format(f_range)
    bandpower.attrs['units'] = 'uV^2/Hz'
    bandpower.attrs['chans'] = chans
    bandpower.attrs['freq_range'] = f_range
    
    return bandpower

## Cortex

In [103]:
cx_chans = channel_groups.superficial_ctx[SUBJECT]
cx_sig = load_timeseries(bin_path, cx_chans, start_time=START_TIME, end_time=END_TIME, xarray=True)

nChan: 385, nFileSamp: 18000001


In [104]:
cx_spg = get_spectrogram(cx_sig)

In [105]:
cx_theta = _get_bandpower(cx_spg, theta_range, cx_chans)
cx_delta = _get_bandpower(cx_spg, delta_range, cx_chans)

### Plot

In [36]:
plot_spectrogram(cx_spg.frequency, cx_spg.time, cx_spg, yscale='log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
_, ax = plt.subplots(figsize=(24, 3))
plot_time_series(cx_spg.time.values, cx_delta.values, ax=ax, title="Cortical Delta", ylabel="Power")
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
_, ax = plt.subplots(figsize=(24, 3))
plot_time_series(cx_spg.time.values, cx_theta.values, ax=ax, title="Cortical Theta", ylabel="Power")
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
_, ax = plt.subplots(figsize=(24, 3))
plot_time_series(cx_spg.time.values, cx_theta.values / cx_delta.values, title="Cortical theta/delta ratio", ylabel="Ratio", ax=ax)
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Theta

In [106]:
sr_chans = channel_groups.stratum_radiatum_140um_to_200um[SUBJECT]
sr_sig = load_timeseries(bin_path, sr_chans, start_time=START_TIME, end_time=END_TIME, xarray=True)

nChan: 385, nFileSamp: 18000001


In [107]:
sr_spg = get_spectrogram(sr_sig)

In [108]:
sr_theta = _get_bandpower(sr_spg, theta_range, sr_chans)
sr_delta = _get_bandpower(sr_spg, delta_range, sr_chans)

### Plot

In [43]:
plot_spectrogram(sr_spg.frequency, sr_spg.time, sr_spg, yscale='log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
_, ax = plt.subplots(figsize=(20, 3))
plot_time_series(sr_spg.time.values, sr_delta.values, title="SR Delta", ylabel="Power", ax=ax)
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
_, ax = plt.subplots(figsize=(20, 3))
plot_time_series(sr_spg.time.values, sr_theta.values, title="SR Theta", ylabel="Power", ax=ax)
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:
_, ax = plt.subplots(figsize=(20, 3))
plot_time_series(sr_spg.time.values, sr_theta.values / sr_delta.values, title="SR theta/delta ratio", ylabel="Ratio", ax=ax)
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
_, ax = plt.subplots(figsize=(20, 3))
plot_time_series(sr_spg.time.values, np.log10(sr_theta.values) / np.log10(sr_delta.values), title="SR log(theta)/log(delta) ratio", ylabel="Ratio", ax=ax)
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
_, ax = plt.subplots(figsize=(24, 3))
plot_time_series(sr_spg.time.values, gaussian_smooth(sr_theta.values / sr_delta.values, smoothing_sigma, 1 / np.diff(sr_spg.time.values).mean()), title="Smoothed SR theta/delta ratio", ylabel="Ratio", ax=ax)
#plot_hypnogram_overlay(hypno, ax=ax)

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Save

In [109]:
bandpower = xr.Dataset({'mpta_delta': cx_delta, 'mpta_theta': cx_theta, 'sr_delta': sr_delta, 'sr_theta': sr_theta})

In [110]:
bandpower.to_netcdf(paths.get_datapath(subject=SUBJECT, condition=CONDITION, data="lfp_bandpower.nc"))

In [111]:
bandpower.close()